In [ ]:
import os
import sys
import json
import gzip
import glob
import itertools
import scipy.stats
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import numba as nb
from numba import njit, jit
from scipy.stats import multinomial
from itertools import permutations 
from tqdm import tqdm
from copy import deepcopy
from functools import partial
from argparse import Namespace
from collections import OrderedDict
from functools import partial
from collections import Counter
from multiprocessing import Pool
from scipy.optimize import minimize
from typing import Union
from textwrap import wrap
from ticodm.utils import *
from ticodm.spatial_interaction_model import ProductionConstrainedSIM,TotalConstrainedSIM,instantiate_sim
from ticodm.contingency_table_mcmc import ContingencyTableMarkovChainMonteCarlo
from ticodm.contingency_table import instantiate_ct
from ticodm.markov_basis import instantiate_markov_basis
from ticodm.sim_models.TotalConstrained import *

mpl.rcParams['agg.path.chunksize'] = 10000

# AUTO RELOAD EXTERNAL MODULES
%load_ext autoreload
%autoreload 2

In [ ]:
# Specify seed
seed = 1234
# Specify dimensions
I = 2
J = 3
# Specify total
N = 5000
# Specify noise level
gamma = 10000.0
# Specify maximum beta
max_beta = 50000
# Sample range
min_val = 1
max_val = 500

In [ ]:
print(f'The MCMC on table space will take {(N/2)**2} steps to converge.')

# Debugging

In [ ]:
cm = np.loadtxt("../tests/test_fixtures/cost_matrix_2x3.txt")
lda = np.loadtxt("../tests/test_fixtures/log_destination_attraction_2x3.txt")
od = np.loadtxt("../tests/test_fixtures/origin_demand_2x3.txt")
tab = np.loadtxt("../tests/test_fixtures/table_2x3_n100.txt")

sim_default_config = {
    "cost_matrix":cm,
    "log_destination_attraction":lda,
    "origin_demand":od,
    "table_distribution_name":"multinomial"
}

tc = TotalConstrainedSIM(sim_default_config)

In [ ]:
wksp = np.zeros((2,3))
for i in range(2):
    for j in range(3):
        wksp[i,j] = tc.log_destination_attraction[j]*0.8-0.5*tc.cost_matrix[i,j]

norm = np.sum(np.exp(wksp))
total = 100
flows = np.zeros((2,3))
for i in range(2):
    for j in range(3):
        flows[i,j] = np.exp(wksp[i,j])*total/norm


In [ ]:
logsumexp(wksp)

In [ ]:
np.log(flows.sum())

In [ ]:
np.log(flows)

In [ ]:
tc.log_intensity(tc.log_destination_attraction,np.array([0.8,0.5]),100)

# Create synthetic flows based on parameters

In [ ]:
# Fix random seed
np.random.seed(seed)

# Construct rowsums
rowsum_probs = np.random.uniform(0,1,I)
for i in range(3):
    index = np.random.randint(I)
    if i == 0:
        rowsum_probs[index] = np.random.uniform(0.8,1.2)
    if i == 1:
        rowsum_probs[index] = np.random.uniform(1.2,1.4)
    if i == 3:
        rowsum_probs[index] = np.random.uniform(1.4,1.6)    
rowsum_probs = rowsum_probs/np.sum(rowsum_probs)
rowsums = np.random.multinomial(n=N,pvals=rowsum_probs)
while 0 in rowsums:
    rowsums = np.random.multinomial(n=N,pvals=rowsum_probs)

# These are temporary - they will be used to define cost matrix column probabilities
colsum_probs = np.random.randint(2,5,size=J)
colsum_probs = colsum_probs/np.sum(colsum_probs)
colsums = np.random.multinomial(n=N,pvals=colsum_probs)
while 0 in colsums:
    colsums = np.random.multinomial(n=N,pvals=colsum_probs)

# Construct cost matrix
cost_matrix = np.diag(np.random.randint(1,5,size=(min(I,J))))
if I == J:
    for i in range(I):
        temp_probs = colsum_probs # Replace with np.random.uniform(0,1,J)
        sample = np.random.multinomial(n=max_val,pvals=temp_probs/np.sum(temp_probs))
        cost_matrix[i,:] = sample
#         cost_matrix[i,i] = 1
        cost_matrix[:,i] = cost_matrix[i,:]
#     cost_matrix = np.triu(cost_matrix).T+np.triu(cost_matrix)-np.diag(np.diag(cost_matrix))
else:
    cost_matrix = np.random.randint(1,5,size=(I,J))
    
total_cost = np.sum(cost_matrix)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(cost_matrix/np.sum(cost_matrix), cmap=plt.cm.coolwarm, interpolation='nearest')
plt.title('Cost matrix',fontsize=16)
plt.ylabel('Origins',fontsize=16)
plt.yticks(range(I))
plt.xlabel('Destinations',fontsize=16)
plt.xticks(range(J))
plt.colorbar(fraction=0.046, pad=0.04)
plt.show()

In [ ]:
dummy_config = Namespace(**{'settings':{'logging': 'info',
 'inputs': {'seed': seed,
  'generate_data': True,
  'dataset': '../data/inputs/synthetic_2x3_N_100',
  'contingency_table': {'ct_type': 'ContingencyTable2D',
   'dependence_model': True,
    'import':{
      'true_table':'table.txt',
    'rowsums':'rowsums.txt',
    'colsums':'colsums.txt'
    },
   'sampler': 'monte_carlo_sample',
    'I': I,
    'J': J,
    'rowsums': rowsums.astype('int32'),
    'colsums': colsums.astype('int32'),
    'diagonal': False,
    'sample_range': [1, 2000]}},
    "spatial_interaction_model":{'sim_type': 'TotalConstrained',
 'beta_max': max_beta,
 'epsilon': 1.0,
 'gamma': gamma,
#  'delta': 0.3/J,
 'noise_percentage': 3,
 'generate': {'cost_matrix': cost_matrix,
              'origin_demand': rowsums/np.sum(rowsums).astype('float32'),
              'delta':0.012,
              'kappa':1.396,
             'alpha_true': 0.8,#1.7,
             'beta_true': 0.4},#0.02}
'import':{
    'log_destination_attraction':'log_destination_attraction.txt',
    'origin_demand':'rowsums.txt',
    'cost_matrix':'cost_matrix.txt',
    'true_log_destination_attraction':'true_log_destination_attraction.txt',
    'true_theta':'true_theta.txt'
    }}},
 "mcmc":{
     "contingency_table":{
         "table_steps":1,
         "proposal":"direct_sampling",
         "table0":"monte_carlo_sample",
         "margin0":'multinomial'
     }
 },
 "outputs":{"directory":"../data/outputs/"}
})
# ct = instantiate_ct(dummy_config)
sim = instantiate_sim(dummy_config)
# ct_mcmc = ContingencyTableMarkovChainMonteCarlo(ct)

# Create synthetic table

In [ ]:
# Store true theta as array
theta_true = np.array([sim.alpha_true,sim.beta_true*sim.bmax,sim.delta,sim.gamma,sim.kappa,sim.epsilon])

In [ ]:
destination_attraction = np.exp(sim.log_true_destination_attraction)/np.sum(np.exp(sim.log_true_destination_attraction))
fig1, ax1 = plt.subplots(figsize=(8,8))
ax1.set_title('Proportion of destination attraction by zone')
ax1.pie(destination_attraction, labels=range(J), autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.show()

In [ ]:
destination_attraction_data = np.exp(sim.log_destination_attraction)/np.sum(np.exp(sim.log_destination_attraction))
fig1, ax1 = plt.subplots(figsize=(8,8))
ax1.set_title('Proportion of destination attraction data by zone')
ax1.pie(destination_attraction, labels=range(J), autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.show()

In [ ]:
def my_multinomial_mode(n,p,MAX_ITERATIONS:int=10000):
    # Implementing algorithm from "Determination of the modes of a Multinomial distribution"
    # Get length of probability vector
    r = np.shape(p)[0]
    # Make make first towards finding mode
    kis = np.floor((n+r/2)*p).astype('int32')
    # Sum all elements to check if they sum up to n
    n0 = int(np.sum(kis))
    # Generate random vector in [0,1]^r
    fis = np.random.uniform(0,1,r)
    # Keep iterations counter
    counter = 0
    try:
        if n0 < n:
            qis = np.divide(1-fis,kis+1)
            while n0 < n:
                min_index = np.argmin(qis)
                kis[min_index] += 1
                fis[min_index] -= 1
                qis[min_index] = (1-fis[min_index])/(kis[min_index]+1)
                n0 += 1
                counter += 1
                if counter > MAX_ITERATIONS:
                    raise ValueError()
        if n0 > n:
            # Compute qis
            qis = np.divide(fis,kis)
            while n0 > n:
                min_index = np.argmin(qis)
                kis[min_index] -= 1
                fis[min_index] += 1
                qis[min_index] = fis[min_index]/kis[min_index]
                n0 -= 1
                counter += 1
                if counter > MAX_ITERATIONS:
                    raise ValueError()
        
        return kis.astype('int32')

    except:
        # Find mode through search
        print(f'Find mode through search, n = {n}, p = {len(p)}')
        support = [x for x in permutations(range(0,n),len(p)) if sum(x) == n]
        mode = np.zeros(len(p))
        max_prob = 0
        for v in support:
            if scipy.stats.multinomial.pmf(v,n=n,p=p) > max_prob:
                max_prob = scipy.stats.multinomial.pmf(v,n=n,p=p)
                mode = v
        return mode.astype('int32')


In [ ]:
# Construct true table by taking mode of multinomial distribution for each row
true_table = np.zeros((sim.I,sim.dims[1]),dtype='int32')
# Get table intensities
log_intensities = sim.log_intensity(sim.log_true_destination_attraction,theta_true)
# Loop each row and extract mode of multinomial distribution
for i in tqdm(range(ct.I)):
    # Get row probabilities
    row_log_probs = log_intensities[i,:] - logsumexp(log_intensities[i,:])
#     print('np.exp(row_log_probs)',np.exp(row_log_probs))
#     print('ct.rowsums[i]',ct.rowsums[i])
    true_row = my_multinomial_mode(ct.rowsums[i],np.exp(row_log_probs))
    true_row_log_prob = -np.infty
    # Obtain mode(s) of distribution 100 times and keep the one with the higher probability
    # This is done because the mode extraction algorithm does not guarantee uniqueness
    for j in range(10):
        # Obtain mode of distribution
        mode = my_multinomial_mode(ct.rowsums[i],np.exp(row_log_probs))
#         print('mode',mode)
        if multinomial.logpmf(x=mode,n=ct.rowsums[i],p=np.exp(row_log_probs)) > true_row_log_prob:
            true_row_log_prob = multinomial.logpmf(x=mode,n=ct.rowsums[i],p=np.exp(row_log_probs))
            # Update row
            true_row = mode
    # Store true row
    true_table[i,:] = true_row
    
# Update colsums based on mode
ct.update_colsums(true_table.sum(axis=0))
# Update table randomly
ct.table = true_table #ct.table_monte_carlo_sample()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.exp(log_intensities), cmap=plt.cm.coolwarm, interpolation='nearest')
for (j,i),label in np.ndenumerate(np.exp(log_intensities)):
    plt.text(i,j,int(label),ha='center',va='center')
    plt.text(i,j,int(label),ha='center',va='center')
plt.title('Intensities',fontsize=16)
plt.xlabel('Destinations',fontsize=16)
plt.xticks(range(J))
plt.ylabel('Origins',fontsize=16)
plt.yticks(range(I))
plt.colorbar(fraction=0.046, pad=0.04)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(ct.table, cmap=plt.cm.coolwarm, interpolation='nearest')
for (j,i),label in np.ndenumerate(ct.table):
    plt.text(i,j,label,ha='center',va='center')
    plt.text(i,j,label,ha='center',va='center')
plt.title('Table',fontsize=16)
plt.xlabel('Destinations',fontsize=16)
plt.xticks(range(J))
plt.ylabel('Origins',fontsize=16)
plt.yticks(range(I))
plt.colorbar(fraction=0.046, pad=0.04)
plt.show()

# ODE Iterative Solver

In [ ]:
# cm = deepcopy(sim.cost_matrix)
# od = deepcopy(sim.origin_demand)
np.random.shuffle(sim.cost_matrix)
np.random.shuffle(sim.origin_demand)

In [ ]:
solution = ode_stationary_points_iterative_solver(
                    theta_true,
                    sim.origin_demand,
                    sim.cost_matrix,
                    np.log(1/sim.dims[1])*np.ones(sim.dims[1]),
                    convergence_threshold=1e-10
)

In [ ]:
plt.figure(figsize=(10,10))
for j in range(sim.dims[1]):
    plt.plot(range(solution.shape[0]),np.exp(solution[:,j]))
plt.ylim(sim.delta/sim.kappa-0.01,min(1.0,np.max(np.exp(solution))*10/9))
plt.xticks(range(solution.shape[0])[::10])
plt.ylabel(r'Equilibrium $W_j$',fontsize=14)
plt.xlabel('Iteration number',fontsize=14)
_=plt.plot(range(solution.shape[0]),np.ones(solution.shape[0])*(sim.delta/sim.kappa),color='red')

In [ ]:
np.exp(solution[-1])

In [ ]:
np.exp(sim.log_true_destination_attraction)

# Euler Mariyama solver

In [ ]:
num_runs = 1#000
xs_ensemble = []
for i in tqdm(range(num_runs)):
    ts,xs = sde_solver(t0=1,
                   t1=50000,
                   N=100000,
                   x0=np.ones(J)*np.log(1/J),
                   theta=theta_true,
                   origin_demand=sim.origin_demand,
                   cost_matrix=sim.cost_matrix,
                   break_early=True,
                   break_threhshold=1e-5)
    if i == 0:
        mean_xs = xs
    else:
        mean_xs = (1/(i+1))*xs + (i/(i+1))*mean_xs

In [ ]:
# Plot mean
plt.figure(figsize=(10,10))
for j in range(mean_xs.shape[1]):
    plt.plot(ts,mean_xs[:,j])
plt.xlabel("time")
plt.ylabel("x")
plt.show()

In [ ]:
destination_attraction = np.exp(sim.log_true_destination_attraction)/np.sum(np.exp(sim.log_true_destination_attraction))
fig1, ax1 = plt.subplots(figsize=(8,8))
ax1.set_title('Proportion of destination attraction by zone')
ax1.pie(destination_attraction, labels=range(J), autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.show()

In [ ]:
destination_attraction_solution = np.exp(mean_xs[-1])/np.sum(np.exp(mean_xs[-1]))
fig1, ax1 = plt.subplots(figsize=(8,8))
ax1.set_title('Proportion of destination attraction SDE solution by zone')
ax1.pie(destination_attraction_solution, labels=range(J), autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.show()

In [ ]:
smallI = 2
smallJ = 3

In [ ]:
# Compare with this
large_table_w_sol = deepcopy(np.exp(solution[-1]))
large_table_o = deepcopy(sim.origin_demand)
large_table_total = deepcopy(sim.total_flow)
large_table_cost = deepcopy(sim.cost_matrix)
large_table_intensity = np.exp(sim.log_intensity(np.log(large_table_w_sol),[sim.alpha_true,sim.beta_true*sim.bmax]))

large_table_margin_w_sol = (large_table_w_sol[:smallJ]/(1-large_table_w_sol[smallJ:].sum()))
large_table_margin_cost = large_table_cost[:smallI,:smallJ]
large_table_margin_intensity = large_table_intensity[:smallI,:smallJ]
large_table_margin_o = large_table_margin_intensity[:smallI,:smallJ].sum(axis=1)

In [ ]:
smaller_solution = ode_stationary_points_iterative_solver(
                    theta_true,
                    large_table_margin_o/large_table_margin_o.sum(),
                    large_table_margin_cost/large_table_margin_cost.sum(),
                    np.log(1/smallJ)*np.ones(smallJ),
                    convergence_threshold=1e-10
)

small_table_w_sol = deepcopy(np.exp(smaller_solution[-1]))
smaller_sim = deepcopy(sim)
smaller_sim.cost_matrix = large_table_margin_cost/large_table_margin_cost.sum()
smaller_sim.origin_demand = large_table_margin_o/large_table_margin_o.sum()
smaller_sim.total_flow = large_table_margin_o.sum()
smaller_sim.I = smallI
smaller_sim.dims[1] = smallJ
small_table_intensity = np.exp(smaller_sim.log_intensity(smaller_solution[-1],[sim.alpha_true,sim.beta_true*sim.bmax]))

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(large_table_margin_intensity, cmap=plt.cm.coolwarm, interpolation='nearest')
for (j,i),label in np.ndenumerate(large_table_margin_intensity):
    plt.text(i,j,int(label),ha='center',va='center')
    plt.text(i,j,int(label),ha='center',va='center')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(small_table_intensity, cmap=plt.cm.coolwarm, interpolation='nearest')
for (j,i),label in np.ndenumerate(small_table_intensity):
    plt.text(i,j,int(label),ha='center',va='center')
    plt.text(i,j,int(label),ha='center',va='center')
plt.show()

# Plot demand size curves

In [ ]:
def ode_log_stationary_points_partial_update(fixed_index,xx,theta,origin_demand,cost_matrix):
    # Get dimensions of cost matrix
    _,ncols = np.shape(cost_matrix)
    alpha = theta[0]
    kappa = theta[4]
    delta = theta[2]

    # Compute lambdas
    log_lambdas = log_flow_matrix(np.asarray(xx),np.asarray(theta),origin_demand,cost_matrix,1.0)
    # Get log stationary points
    log_stationary_points = np.log((np.sum(np.exp(log_lambdas),axis=0) + delta)) - np.log(kappa)
    log_stationary_points[fixed_index] = xx[fixed_index]
    
    return log_stationary_points

def ode_stationary_points_partial_iterative_solver(fixed_index,fixed_value,theta,origin_demand,cost_matrix,convergence_threshold:float=1e-9):
    # Extract necessary data
    delta = theta[2]
    _,ncols = np.shape(cost_matrix)
    # Start with the same attraction for every zone
    log_destination_attraction_prev = np.log(delta)*np.ones(ncols)
    log_destination_attraction_prev[fixed_index] = np.log(fixed_value)
    # Perform update for stationary points
    log_destination_attraction_new = ode_log_stationary_points_partial_update(fixed_index,log_destination_attraction_prev,theta,origin_demand,cost_matrix)
    log_destination_attractions = log_destination_attraction_prev.reshape(1,np.shape(log_destination_attraction_prev)[0])
    # Solve equation until successive solutions do not change signficantly (equilibrium point identified)
    while (np.absolute(log_destination_attraction_new-log_destination_attraction_prev) > convergence_threshold).any():
        log_destination_attraction_prev = log_destination_attraction_new
        log_destination_attraction_prev[fixed_index] = np.log(fixed_value)
        log_destination_attraction_new = ode_log_stationary_points_partial_update(fixed_index,log_destination_attraction_prev,theta,origin_demand,cost_matrix)
        log_destination_attractions = np.append(log_destination_attractions,log_destination_attraction_new.reshape(1,np.shape(log_destination_attraction_new)[0]),axis=0)

    return log_destination_attractions

In [ ]:
fig,axs = plt.subplots(nrows=int(J//3),ncols=3,figsize=(30,10))
for j in range(sim.dims[1]):
    xxs = deepcopy(solution[-1])
    size = np.empty(100)
    demand = np.empty(100)
    for i,val in enumerate(np.linspace(1e-10,1.0,100)):
        xxs = ode_stationary_points_partial_iterative_solver(j,val,theta_true,sim.origin_demand,sim.cost_matrix,convergence_threshold=1e-9)[-1]
        size[i] = np.exp(xxs[j])
        demand[i] = np.exp(log_flow_matrix(xxs,theta_true,sim.origin_demand,sim.cost_matrix,1.0)).sum(axis=0)[j]
    if sim.dims[1] <= 3:
        axs[j].plot(size,demand,label='Demand')
        axs[j].set_xlabel(r'$W_j$',fontsize=16,labelpad=-5)
        axs[j].set_ylabel(r'$D_j$',rotation=0,labelpad=10,fontsize=16)
        axs[j].set_title(f'Destination {j}')
        axs[j].plot(size,sim.kappa*size,label='Cost')
        axs[j].legend()
    else:
        axs[int(j//3),j%3].plot(size,demand,label='Demand')
        axs[int(j//3),j%3].set_xlabel(r'$W_j$',fontsize=16,labelpad=-5)
        axs[int(j//3),j%3].set_ylabel(r'$D_j$',rotation=0,labelpad=10,fontsize=16)
        axs[int(j//3),j%3].set_title(f'Destination {j}')
        axs[int(j//3),j%3].plot(size,sim.kappa*size,label='Cost')
        axs[int(j//3),j%3].legend()

# Export to file

In [ ]:
# ct.table = ct.table_monte_carlo_sample()

In [ ]:
ct.export(f"../data/inputs/synthetic_{I}x{J}_N_{N}/",overwrite=True)

In [ ]:
sim.export(f"../data/inputs/synthetic_{I}x{J}_N_{N}/",overwrite=True)